Experimento 13 - Linear Interpolation Imputer, MAD outlier detection, min max normalization, knn classifier

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
!pip install --quiet ucimlrepo
from ucimlrepo import fetch_ucirepo

In [2]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [3]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [4]:
import random

#Introducing Missing values (5%)
x = df.copy()

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      x[i][jndex] = np.NaN

df = x

<ipython-input-4-fbb7389abead>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN


In [5]:
df.isna().sum()

Area               691
Perimeter          713
MajorAxisLength    660
MinorAxisLength    681
AspectRatio        716
Eccentricity       632
ConvexArea         676
EquivDiameter      676
Extent             735
Solidity           678
Roundness          670
Compactness        618
ShapeFactor1       674
ShapeFactor2       691
ShapeFactor3       676
ShapeFactor4       713
dtype: int64

In [6]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,NaN,28715.0,190.141097,NaN,0.988856,NaN,NaN,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,NaN,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,NaN,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,NaN,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,NaN,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,NaN,0.916603,0.801865,0.006858,0.001749,0.642988,NaN
13607,42101.0,757.499,281.576392,190.713136,NaN,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,NaN,0.006688,0.001886,0.676099,0.998219
13608,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [7]:
#Imputing Missing Values with Linear Interpolation
df_imputed = df.interpolate(method="nearest", order=3, limit=None,
                            limit_direction='both').ffill().bfill()
df_imputed

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,0.411785,28715.0,190.141097,0.783968,0.988856,0.887034,0.953861,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,173.888747,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.908774,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,182.516516,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003215,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990862,0.916603,0.801865,0.006858,0.001749,0.642988,0.998176
13607,42101.0,757.499,281.576392,190.713136,1.552728,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.801865,0.006688,0.001886,0.676099,0.998219
13608,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [8]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

          Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count     count           count           count       count   
Class                                                                  
BARBUNYA  1322      1322            1322            1322        1322   
BOMBAY     522       522             522             522         522   
CALI      1630      1630            1630            1630        1630   
DERMASON  3546      3546            3546            3546        3546   
HOROZ     1928      1928            1928            1928        1928   
SEKER     2027      2027            2027            2027        2027   
SIRA      2636      2636            2636            2636        2636   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                                      
BARBUNYA         1322       1322          1322   13

In [9]:
#Outlier Removal with MAD
from scipy import stats

print(f'Pre Outlier Shape: {df_imputed.shape}')

df_no_outliers = df_imputed.copy()
print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      mad = 1.4826 * np.median(np.absolute(class_unique[feature] - class_unique[feature].median()))
      upper = class_unique[feature].median() + (3 * mad)
      lower = class_unique[feature].median() - (3 * mad)
      excluded_lower = pd.Series(class_unique[class_unique[feature] < lower].index)
      excluded_upper = pd.Series(class_unique[class_unique[feature] > upper].index)
      df_no_outliers.drop(excluded_lower.values, inplace = True, errors='ignore')
      df_no_outliers.drop(excluded_upper.values, inplace = True, errors='ignore')


print(f'Pos Outlier Shape: {df_no_outliers.shape}')

Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
Pos Outlier Shape: (12142, 17)


In [10]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1189  1189      1189            1189            1189        1189   
BOMBAY     472   472       472             472             472         472   
CALI      1513  1513      1513            1513            1513        1513   
DERMASON  3203  3203      3203            3203            3203        3203   
HOROZ     1614  1614      1614            1614            1614        1614   
SEKER     1754  1754      1754            1754            1754        1754   
SIRA      2397  2397      2397            2397            2397        2397   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [11]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
1,30279.0,634.927,212.560556,181.510182,1.171067,0.520401,30600.0,196.347702,0.775688,0.989510,0.943852,0.923726,0.007020,0.003153,0.853270,0.999236,SEKER
2,30519.0,670.033,212.996755,182.737204,1.165591,0.513760,30847.0,197.124320,0.770682,0.989367,0.967109,0.925480,0.006979,0.003158,0.856514,0.998345,SEKER
3,30685.0,631.934,213.534145,183.157146,1.165852,0.514081,31044.0,197.659696,0.771561,0.988436,0.954240,0.925658,0.006959,0.003152,0.856844,0.998953,SEKER
4,30834.0,631.934,217.227813,180.897469,1.200834,0.553642,31120.0,198.139012,0.783683,0.990810,0.970278,0.912125,0.007045,0.003008,0.831973,0.999061,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12137,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990862,0.916603,0.801865,0.006858,0.001749,0.642988,0.998176,DERMASON
12138,42101.0,757.499,281.576392,190.713136,1.552728,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.801865,0.006688,0.001886,0.676099,0.998219,DERMASON
12139,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
12140,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [12]:
#Normalization with min max
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,0.041124,0.068688,0.043529,0.136911,0.124544,0.325752,0.040555,0.082411,0.705834,0.794149,0.866700,0.842879,0.656551,0.870018,0.810867,0.967425,SEKER
1,0.045271,0.076452,0.043045,0.155099,0.095851,0.243137,0.044693,0.089975,0.697736,0.792180,0.854259,0.890723,0.622647,0.906817,0.865137,0.975718,SEKER
2,0.046378,0.101649,0.043839,0.159099,0.091786,0.230174,0.045816,0.091976,0.681015,0.786432,0.926634,0.896337,0.616450,0.908746,0.871564,0.932152,SEKER
3,0.047144,0.074304,0.044818,0.160468,0.091980,0.230801,0.046712,0.093355,0.683953,0.748987,0.886586,0.896907,0.613384,0.906386,0.872217,0.961894,SEKER
4,0.047831,0.074304,0.051543,0.153101,0.117952,0.308019,0.047058,0.094590,0.724434,0.844458,0.936495,0.853602,0.626435,0.856113,0.822954,0.967181,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12137,0.099780,0.166003,0.181715,0.169555,0.379212,0.720563,0.098845,0.180563,0.493632,0.846541,0.769465,0.500779,0.598174,0.415091,0.448619,0.923918,DERMASON
12138,0.099799,0.164426,0.168705,0.185100,0.379212,0.663374,0.098782,0.180592,0.778739,0.842120,0.786307,0.500779,0.572370,0.462993,0.514204,0.925998,DERMASON
12139,0.099974,0.165734,0.168639,0.186648,0.319708,0.660177,0.099123,0.180861,0.544924,0.807821,0.775132,0.567544,0.571325,0.463847,0.515760,0.855050,DERMASON
12140,0.100011,0.168933,0.171994,0.183674,0.332140,0.673821,0.099568,0.180918,0.462956,0.723926,0.742402,0.550673,0.577755,0.451149,0.498631,0.779534,DERMASON


In [14]:
#Classification with Knn-Classification

from sklearn.neighbors import KNeighborsClassifier as knnc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()

classifier = knnc(n_neighbors=10)
x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}

y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)


print('Experimento 13')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')

Experimento 13
Acurácia Média: 97.27%
Precisão Média: 95.00%
Revocação Média: 93.77%
F1-Score Médio: 93.52%
